In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [2]:
!pip install pytorch_tabnet

  Obtaining dependency information for pytorch_tabnet from https://files.pythonhosted.org/packages/0f/92/ed98b89b7cf5661656daa4cc88e578f712eb5eae41b8f46a56c1ece3a895/pytorch_tabnet-4.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.7 MB/s eta 0:00:00


In [4]:
# Load dataset
df = pd.read_csv("heart_disease_uci.csv")
df.drop(columns=["id", "dataset"], inplace=True)
df["target"] = df["num"].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=["num"], inplace=True)

numerical = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
categorical = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

# Handle missing values and encode categorical features
df[numerical] = SimpleImputer(strategy="median").fit_transform(df[numerical])
for col in categorical:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))
df[numerical] = StandardScaler().fit_transform(df[numerical])

X = df[numerical + categorical].values
y = df["target"].values


In [5]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# TabNet Classifier
tabnet = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    momentum=0.3, 
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=0,
    seed=42
)

In [6]:
# Fit TabNet
tabnet.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_name=['val'],
    eval_metric=['auc'],
    max_epochs=200,
    patience=20,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# Predictions
y_pred_proba = tabnet.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int)


Early stopping occurred at epoch 69 with best_epoch = 49 and best_val_auc = 0.90423


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [7]:
# Metrics
results = {
    'Model': "TabNet",
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba)
}

print("\n📊 Final Evaluation Metrics for TabNet:")
for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")



📊 Final Evaluation Metrics for TabNet:
Model: TabNet
Accuracy: 0.8533
F1 Score: 0.8643
Precision: 0.8866
Recall: 0.8431
ROC AUC: 0.9042


# attempt 2

In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [2]:
# Load and preprocess dataset
df = pd.read_csv("heart_disease_uci.csv")
df.drop(columns=["id", "dataset"], inplace=True)
df["target"] = df["num"].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=["num"], inplace=True)

numerical = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
categorical = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

df[numerical] = SimpleImputer(strategy="median").fit_transform(df[numerical])
for col in categorical:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))
df[numerical] = StandardScaler().fit_transform(df[numerical])

X_full = df[numerical + categorical].values
y_full = df["target"].values

# Feature Selection using Mutual Information
selector = SelectKBest(mutual_info_classif, k=10)
X = selector.fit_transform(X_full, y_full)
y = y_full

In [3]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 32, 128),
        "n_a": trial.suggest_int("n_a", 32, 128),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True),
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "optimizer_params": dict(lr=trial.suggest_float("lr", 1e-3, 1e-1, log=True)),
        "n_independent": trial.suggest_int("n_independent", 1, 3),
        "n_shared": trial.suggest_int("n_shared", 1, 3)
    }

    tabnet = TabNetClassifier(
        **params,
        optimizer_fn=torch.optim.Adam,
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=0,
        seed=42
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_t, X_v = X_train[train_idx], X_train[val_idx]
        y_t, y_v = y_train[train_idx], y_train[val_idx]

        tabnet.fit(
            X_t, y_t,
            eval_set=[(X_v, y_v)],
            eval_metric=["auc"],
            max_epochs=200,
            patience=30,
            batch_size=256,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False
        )

        y_pred_proba = tabnet.predict_proba(X_v)[:, 1]
        auc = roc_auc_score(y_v, y_pred_proba)
        aucs.append(auc)

    return np.mean(aucs)

In [4]:
# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
print("\n✅ Best Hyperparameters Found:")
for k, v in best_params.items():
    print(f"{k}: {v}")


[I 2025-05-09 23:49:07,207] A new study created in memory with name: no-name-af889f8b-0184-4efc-9aaa-b28ca5e629d5



Early stopping occurred at epoch 41 with best_epoch = 11 and best_val_0_auc = 0.85735


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 94 with best_epoch = 64 and best_val_0_auc = 0.82308


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 60 with best_epoch = 30 and best_val_0_auc = 0.88571


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 79 with best_epoch = 49 and best_val_0_auc = 0.8786


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-05-10 00:41:15,441] Trial 29 finished with value: 0.8772850175018198 and parameters: {'n_d': 103, 'n_a': 90, 'n_steps': 9, 'gamma': 1.7959386151917167, 'lambda_sparse': 6.403704361953742e-05, 'momentum': 0.3101185821590587, 'lr': 0.014799343820458446, 'n_independent': 2, 'n_shared': 2}. Best is trial 12 with value: 0.8848017314412978.



Early stopping occurred at epoch 123 with best_epoch = 93 and best_val_0_auc = 0.90367

✅ Best Hyperparameters Found:
n_d: 36
n_a: 128
n_steps: 10
gamma: 1.530158700027477
lambda_sparse: 0.00015784720012436288
momentum: 0.39629648784113414
lr: 0.01415008826669103
n_independent: 2
n_shared: 3


In [6]:
# Final Model Training with Best Hyperparameters
final_tabnet = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    momentum=best_params['momentum'],
    n_independent=best_params['n_independent'],
    n_shared=best_params['n_shared'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params={'lr': best_params['lr']},
    scheduler_params={"step_size": 10, "gamma": 0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1,
    seed=42
)


final_tabnet.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric=["auc"],
    max_epochs=300,
    patience=30,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# Evaluation
y_pred_proba = final_tabnet.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int)

/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.69004 | val_0_auc: 0.77846 |  0:00:00s
epoch 1  | loss: 0.80325 | val_0_auc: 0.83226 |  0:00:00s
epoch 2  | loss: 0.98771 | val_0_auc: 0.65555 |  0:00:01s
epoch 3  | loss: 0.92311 | val_0_auc: 0.69201 |  0:00:01s
epoch 4  | loss: 0.95697 | val_0_auc: 0.76309 |  0:00:02s
epoch 5  | loss: 0.89131 | val_0_auc: 0.74271 |  0:00:02s
epoch 6  | loss: 0.78384 | val_0_auc: 0.76327 |  0:00:02s
epoch 7  | loss: 0.66402 | val_0_auc: 0.71353 |  0:00:03s
epoch 8  | loss: 0.7121  | val_0_auc: 0.77499 |  0:00:03s
epoch 9  | loss: 0.54359 | val_0_auc: 0.79962 |  0:00:04s
epoch 10 | loss: 0.5907  | val_0_auc: 0.79352 |  0:00:04s
epoch 11 | loss: 0.6592  | val_0_auc: 0.80906 |  0:00:04s
epoch 12 | loss: 0.63364 | val_0_auc: 0.79675 |  0:00:05s
epoch 13 | loss: 0.90621 | val_0_auc: 0.85473 |  0:00:05s
epoch 14 | loss: 0.72469 | val_0_auc: 0.79962 |  0:00:06s
epoch 15 | loss: 0.63839 | val_0_auc: 0.75598 |  0:00:06s
epoch 16 | loss: 0.59319 | val_0_auc: 0.71533 |  0:00:07s
epoch 17 | los

/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [7]:
results = {
    'Model': "TabNet (Optuna Tuned)",
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba)
}

print("\n📊 Final Evaluation Metrics for Tuned TabNet:")
for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


📊 Final Evaluation Metrics for Tuned TabNet:
Model: TabNet (Optuna Tuned)
Accuracy: 0.8261
F1 Score: 0.8545
Precision: 0.7966
Recall: 0.9216
ROC AUC: 0.9065


# Attempt 3

In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [2]:
# Load and preprocess dataset
df = pd.read_csv("heart_disease_uci.csv")
df.drop(columns=["id", "dataset"], inplace=True)
df["target"] = df["num"].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=["num"], inplace=True)

numerical = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
categorical = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

df[numerical] = SimpleImputer(strategy="median").fit_transform(df[numerical])
for col in categorical:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))
df[numerical] = StandardScaler().fit_transform(df[numerical])

X_full = df[numerical + categorical].values
y_full = df["target"].values

# Feature Selection + PCA
selector = SelectKBest(mutual_info_classif, k=10)
X_selected = selector.fit_transform(X_full, y_full)
pca = PCA(n_components=8)
X_pca = pca.fit_transform(X_selected)
X = X_pca
y = y_full

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [3]:
def objective(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 32, 128),
        "n_a": trial.suggest_int("n_a", 32, 128),
        "n_steps": trial.suggest_int("n_steps", 3, 8),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True),
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "optimizer_params": {"lr": trial.suggest_float("lr", 1e-3, 1e-1, log=True)},
        "n_independent": trial.suggest_int("n_independent", 1, 3),
        "n_shared": trial.suggest_int("n_shared", 1, 3)
    }

    tabnet = TabNetClassifier(
        **params,
        optimizer_fn=torch.optim.Adam,
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=0,
        seed=42
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_t, X_v = X_train[train_idx], X_train[val_idx]
        y_t, y_v = y_train[train_idx], y_train[val_idx]

        tabnet.fit(
            X_t, y_t,
            eval_set=[(X_v, y_v)],
            eval_metric=["auc"],
            max_epochs=300,
            patience=30,
            batch_size=256,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False
        )

        y_pred_proba = tabnet.predict_proba(X_v)[:, 1]
        auc = roc_auc_score(y_v, y_pred_proba)
        aucs.append(auc)

    return np.mean(aucs)

In [4]:
# Run Optuna with Pruning to speed up
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50, timeout=1800)  # Try for 50 trials or 30 minutes

best_params = study.best_params
print("\n✅ Best Hyperparameters Found:")
for k, v in best_params.items():
    print(f"{k}: {v}")

# Final Model Training with Best Hyperparameters
final_tabnet = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    momentum=best_params['momentum'],
    n_independent=best_params['n_independent'],
    n_shared=best_params['n_shared'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params={'lr': best_params['lr']},
    scheduler_params={"step_size": 10, "gamma": 0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1,
    seed=42
)

final_tabnet.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric=["auc"],
    max_epochs=300,
    patience=30,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


[I 2025-05-10 16:14:51,332] A new study created in memory with name: no-name-ac1be953-aaec-4823-8c6c-ae85517d6946



Early stopping occurred at epoch 44 with best_epoch = 14 and best_val_0_auc = 0.86641


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 35 with best_epoch = 5 and best_val_0_auc = 0.83959


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-05-10 16:44:59,069] Trial 37 finished with value: 0.858185847074736 and parameters: {'n_d': 55, 'n_a': 64, 'n_steps': 4, 'gamma': 1.5186847105706933, 'lambda_sparse': 5.9248045560499986e-05, 'momentum': 0.03717795059635505, 'lr': 0.03983232658874914, 'n_independent': 1, 'n_shared': 1}. Best is trial 30 with value: 0.8770934625948177.



Early stopping occurred at epoch 42 with best_epoch = 12 and best_val_0_auc = 0.88646

✅ Best Hyperparameters Found:
n_d: 61
n_a: 62
n_steps: 4
gamma: 1.7193370395501062
lambda_sparse: 4.511027199039756e-05
momentum: 0.17499993899825866
lr: 0.048765009320005034
n_independent: 3
n_shared: 3


/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.55833 | val_0_auc: 0.6574  |  0:00:00s
epoch 1  | loss: 0.91919 | val_0_auc: 0.83375 |  0:00:00s
epoch 2  | loss: 0.70383 | val_0_auc: 0.76626 |  0:00:00s
epoch 3  | loss: 0.60083 | val_0_auc: 0.82508 |  0:00:00s
epoch 4  | loss: 0.54666 | val_0_auc: 0.85725 |  0:00:01s
epoch 5  | loss: 0.50282 | val_0_auc: 0.84768 |  0:00:01s
epoch 6  | loss: 0.51263 | val_0_auc: 0.85342 |  0:00:01s
epoch 7  | loss: 0.49411 | val_0_auc: 0.86143 |  0:00:01s
epoch 8  | loss: 0.46969 | val_0_auc: 0.87829 |  0:00:01s
epoch 9  | loss: 0.45801 | val_0_auc: 0.88271 |  0:00:02s
epoch 10 | loss: 0.44245 | val_0_auc: 0.87769 |  0:00:02s
epoch 11 | loss: 0.44012 | val_0_auc: 0.87243 |  0:00:02s
epoch 12 | loss: 0.45155 | val_0_auc: 0.8692  |  0:00:02s
epoch 13 | loss: 0.42552 | val_0_auc: 0.85964 |  0:00:02s
epoch 14 | loss: 0.42485 | val_0_auc: 0.84923 |  0:00:03s
epoch 15 | loss: 0.44834 | val_0_auc: 0.85438 |  0:00:03s
epoch 16 | loss: 0.43591 | val_0_auc: 0.86633 |  0:00:03s
epoch 17 | los

/Users/savalideshmukh/anaconda3/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [5]:
# Final Evaluation with Optimal Threshold
y_pred_proba = final_tabnet.predict_proba(X_test)[:, 1]
best_threshold = 0.5  # You can optimize this further if needed
y_pred = (y_pred_proba > best_threshold).astype(int)

results = {
    'Model': "TabNet (Advanced Tuned)",
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba)
}

print("\n📊 Final Evaluation Metrics for TabNet (Advanced):")
for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


📊 Final Evaluation Metrics for TabNet (Advanced):
Model: TabNet (Advanced Tuned)
Accuracy: 0.8261
F1 Score: 0.8431
Precision: 0.8431
Recall: 0.8431
ROC AUC: 0.9103
